1. 定義：ELMo（Embeddings from Language Models）は、2018年に発表された革新的な言語モデルで、自然言語処理（NLP）において重要な進展をもたらしました。ELMoは、文脈に基づいた単語の埋め込み（ワードエンベディング）を生成するモデルで、従来の静的なワードエンベディングとは異なり、各単語の意味がその文脈に応じて変化するという特徴を持っています。

2. ELMoの特徴
文脈に基づくワードエンベディング: ELMoは、単語を文全体の中でどのように使われているかに基づいてエンベディングを生成します。これにより、同じ単語であっても異なる文脈で異なるベクトル表現を持つことができます。

双方向LSTM: ELMoは、双方向LSTM（Long Short-Term Memory）ネットワークを使用しており、文の前後の情報を取り入れて単語の意味を捉えます。

階層的な表現: モデルの内部で生成される表現は、異なる層からの出力を重ね合わせたものであり、各層が異なるレベルの情報（例えば、文法的特徴や意味論的特徴）を捉えています。

ELMoと他のモデルの違い
静的エンベディングとの違い: 従来のワードエンベディング（例: Word2VecやGloVe）は、各単語に対して固定されたベクトルを与えますが、ELMoは文脈に応じて変化する動的なベクトルを提供します。

BERTとの違い: BERT（Bidirectional Encoder Representations from Transformers）と同様に文脈に基づいたモデルですが、BERTはトランスフォーマーを使用しているのに対して、ELMoはLSTMを使用しています。BERTはさらに高度なマスク言語モデリングを用いているため、より強力な性能を発揮しますが、ELMoはそのシンプルさと有効性で依然として利用価値があります。

3. structure

![elmo](./image/elmo_01.webp)

ELMoモデルの説明

入力層（E1, E2, ..., EN）:図の最下部にあるE1, E2, ..., ENは、入力テキスト中の各単語に対応する初期の単語埋め込み（ワードエンベディング）を表しています。これらの埋め込みは通常、事前に学習された静的な埋め込み（例: Word2VecやGloVe）を使用します。

双方向LSTM層（LSTMレイヤー）:ELMoは、双方向LSTM（Bidirectional LSTM）を使用しており、2つのLSTMレイヤーから構成されています。一方のレイヤーはテキストを前から後ろに処理し、もう一方のレイヤーはテキストを後ろから前に処理します。この双方向LSTMにより、各単語の前後の文脈情報を取得します。
図では、各LSTMレイヤーが前後の文脈を取り込むために、左右に双方向の矢印で表現されています。

![elmo](./image/elmo_02.png)

出力層（T1, T2, ..., TN）:

最上部のT1, T2, ..., TNは、LSTMの出力を表します。これらは文脈に基づいた単語の埋め込みであり、各単語がその周囲の文脈に応じて異なるベクトル表現を持つことを示しています。
![elmo](./image/elmo_03.png)

まとめ
ELMoは、自然言語処理の多くのタスクにおいて従来のモデルよりも優れたパフォーマンスを発揮し、NLPの分野で大きな影響を与えました。特に、文脈に応じて動的に変化するワードエンベディングを提供する点が大きな特徴です。

In [ ]:
import torch
import torch.nn as nn

class ELMo(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(ELMo, self).__init__()
        # 単語埋め込み層
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # 多層双方向LSTM層
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)
        # 線形変換層
        self.linear = nn.Linear(hidden_dim * 2, embedding_dim)
        # 層ごとの重み付け（トレーニング可能なパラメータ）
        self.scalar_mix = nn.Parameter(torch.ones(num_layers))

    def forward(self, x):
        # xの形状: (batch_size, seq_length)
        # 単語埋め込み
        embedded = self.embedding(x)
        # LSTMを通して文脈情報を取得
        lstm_out, _ = self.lstm(embedded)
        # 各層の出力を結合
        layer_outputs = torch.stack([lstm_out[:, :, i * self.hidden_dim:(i + 1) * self.hidden_dim] for i in range(self.num_layers)], dim=-1)
        # スカラー加重平均
        mixed_output = torch.sum(self.scalar_mix.unsqueeze(0).unsqueeze(0) * layer_outputs, dim=-1)
        # 線形変換で出力
        out = self.linear(mixed_output)
        return out

# ハイパーパラメータの設定
vocab_size = 10000   # 語彙のサイズ（例として10000）
embedding_dim = 256  # 埋め込み次元
hidden_dim = 512     # 隠れ層の次元
num_layers = 2       # LSTMの層の数

# モデルのインスタンスを作成
elmo = ELMo(vocab_size, embedding_dim, hidden_dim, num_layers)

# 例としてバッチサイズ2、シーケンス長5のダミー入力
inputs = torch.randint(0, vocab_size, (2, 5))

# モデルのフォワードパスを実行
outputs = elmo(inputs)

print("Outputs shape:", outputs.shape)  # 出力の形状を確認
